In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import numpy
import math

In [ ]:
driver = webdriver.Chrome("/Users/inkoo/Downloads/chromedriver")

In [89]:
import requests
url = 'http://dart.fss.or.kr'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

# Scraping

In [90]:
driver.get('http://dart.fss.or.kr')

In [91]:
# 정기공시 클릭
driver.find_element("xpath","/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[2]/div[1]/ul/li[1]/label").click()
time.sleep(1) # 클릭 후 반응을 고려하여 1초 쉬기

In [92]:
# 사업보고서 클릭
driver.find_element("xpath","/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[2]/div[2]/div[1]/ul/li[1]/span/input").click()

In [93]:
# 검색 클릭
driver.find_element("xpath","/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[3]/a").click()

In [94]:
## start day
# start day text input 창 선택
element = driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[2]/div[2]/div[1]/ul/li[3]/div[1]/span[1]/input")
element.send_keys(8 * Keys.BACKSPACE)
element.send_keys("20210101")

In [95]:
## end day
# end day text input 창 선택
element = driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[2]/div[2]/div[1]/ul/li[3]/div[1]/span[2]/input")
element.send_keys(8 * Keys.BACKSPACE)
element.send_keys("20211231")

In [96]:
# 조회건수
driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]/span/select").click()
time.sleep(1)
driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]/span/select/option[4]").click()
time.sleep(1)

In [97]:
# 검색 클릭
driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[2]/div[2]/div[2]/a[1]").click()

# 사업 내용 중 사업 개요, 주요 제품 및 서비스만 추출

In [116]:


d = {}
t = []
l1 = ['3. 원재료 및 생산설비', '3. 주요 원재료', '3. 주요원재료', '3. 원재료', '3. 생산', '3. 설비', 
     '4. 원재료 및 생산설비', '4. 주요 원재료', '4. 주요원재료', '4. 원재료', '4. 생산', '4. 설비',
     '3. 파생', '3. 매입', '3. 영업', '3. 연결대상', '2. 매출', '3. 매출', '4. 매출', '4. 매출 상황', '4. 매출상황',
     '3. 주요종속', '3. 주요 종속', '3. 수주', '3. 수주 상황', '3. 수주상황', '3. 취급품목별 매입 내역',
     '4. 수주', '4. 수주 상황', '4. 수주상황', '5. 수주', '5. 수주 상황', '5. 수주상황',
     '3. 사업부문별', '3. 자원조달', '4. 자원조달', '5. 자원조달', 
     '3. 신규', '4. 매입', '3) 주요 원재료', '3) 원재료', '라. 주요 원재료', '5. 주요 원재료',
     '3) 주요원재료', '3) 원재료', '라. 주요원재료', '5. 주요원재료', '자. 수주상황']
l2 = [(x[:2] + x[3:]) for x in l1]
l = l1 + l2

for j in range(2388, 3178):
    # 사업보고서 xpath를 통해 사업보고서 url 가지고 오기
    n = (j % 100) + abs(int((j%100)/100 - 1)) * 100
    elm = driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[" + str(n) + "]/td[3]/a")
    href = elm.get_attribute('href')
    
    time.sleep(random.uniform(1,2))

    # input url 
    html = urlopen(href) # 사업보고서 url
    # 사업보고서 url로 반기보고서 html 호출
    bsobjt = BeautifulSoup(html,"html.parser") 
    # 기업명
    title = str(bsobjt.find('title'))
    title = title.split('\n\t')[1].split('/사업')[0]

    if title not in d.keys():
        # 사업의 내용
        body = str(bsobjt.find('head'))
        if '사업의 내용' in body:
            body = body.split('사업의 내용"')[1] # input2
            body = body.split('cnt++')[0]
            body = body.split(';')
            body = [x for x in body if ('=' in x)]
            body = [x.split('=')[1].split('"')[1].split('"')[0] for x in body]

            # 사업의 내용 url 생성
            url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[1] + '&dcmNo=' + body[2] + '&eleId=' + body[3] + '&offset=' + body[4] +'&length=' + body[5] + '&dtd=dart3.xsd'
            # 사업의 내용 html 가지고 오기
            html_final = urlopen(url_final)
            bsobjt_final = BeautifulSoup(html_final,"html.parser")
            desc = ''.join(bsobjt_final.findAll(text=True))
            for x in l:
                if x in desc:
                    if title not in t:
                        t.append(title)
                    desc = desc.split(x)[0]
                    break
            
            desc = desc.replace('\xa0', '').replace('\r', '').replace('\n', '').split('HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"II. ')[1]

            d[title] = desc
        else:
            continue
    if (j % 1000) == 100:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[4]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 200:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[5]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 300:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[6]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 400:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[7]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 500:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[8]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 600:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[9]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 700:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[10]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 800:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[11]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 900:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[12]/a").click()                             
        time.sleep(1)
    if (j % 1000) == 0:
        driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[2]/div[2]/ul/li[13]/a").click()                             
        time.sleep(1)

In [122]:
# 사업 내용 추출 엑셀 파일 저장
df = pd.DataFrame(data = d, index = ['사업 내용 (extracted)'])
df = (df.T)
df.to_excel('2021_extracted.xlsx')

In [3]:
# 엑셀 파일 불러오기
df = pd.read_excel('DART-2021_extracted.xlsx')
df

,회사명,사업 내용 (extracted)
0,3S,사업의 내용1. 사업의개요당사는 반도체 웨이퍼 캐리어 사업과 환경시험장치 사업을 주...
1,AJ네트웍스,사업의 내용조직도1. 사업의 개요가. 업계의 현황소유를 중시하던 전통적인 자산 개념...
2,AK홀딩스,사업의 내용1. 사업의 개요지주사업 부문(1) 업계의 현황1) 산업의 특성 등① 산...
3,APS홀딩스,사업의 내용1. 사업의 개요가. 업계의 현황지주회사란 다른 회사의 주식소유를 목적으...
4,AP시스템,사업의 내용1. 사업의 개요가. 산업별 현황 [OLED 장비산업] 1) 산업의 특성...
...,...,...
2827,흥국화재,사업의 내용1. 사업의 개요가. 업계의 현황(1) 산업의 특성 및 성장성- 손해보험...
2828,흥아해운,사업의 내용1. 사업의 개요가. 업계의 현황(1) 해운업의 특성해운업은 선박의 공간...
2829,희림,사업의 내용1.사업의 개요가. 산업의 특성당사는 건설 관련 서비스업인 건축물의 설계...
2830,힐세리온,사업의 내용1. 사업의 개요 주식회사 힐세리온은 핸드핼드 초음파 진단기(Sonon)...


# 앤디포스 (구분 X)

In [388]:
# 사업보고서 xpath를 통해 사업보고서 url 가지고 오기
elm = driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[2]/td[3]/a")
href = elm.get_attribute('href')

# input url 
html = urlopen(href) # 사업보고서 url
print(href)

https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20211228000497


In [389]:
# 사업보고서 url로 반기보고서 html 호출
bsobjt = BeautifulSoup(html,"html.parser") 
# 기업명
title = str(bsobjt.find('title'))
title = title.split('\n\t')[1].split('/사업')[0]
# 사업의 내용
body = str(bsobjt.find('head'))
body = body.split('사업의 내용"')[1] # input2
body = body.split('cnt++')[0]
body = body.split(';')
body = [x for x in body if ('=' in x)]
body = [x.split('=')[1].split('"')[1].split('"')[0] for x in body]

In [390]:
# 사업내용 url 생성
url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[1] + '&dcmNo=' + body[2] + '&eleId=' + body[3] + '&offset=' + body[4] +'&length=' + body[5] + '&dtd=dart3.xsd'

In [391]:
# 사업내용 html 가지고 오기
html_final = urlopen(url_final)
bsobjt_final = BeautifulSoup(html_final,"html.parser")

In [392]:
bsobjt_final

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-1"><a name="toc1">II. 사업의 내용</a></p>
<p><br/><span style="font-size:16pt;line-height:1.6em;color:#0012FF;font-weight:bold;">1. 사업의 개요<br/></span><br/>
</p>
<p><span style="font-weight:bold;">가. 산업의 특성<br/><br/>(1) 휴대폰 등 모바일기기용 양면 테이프<br/></span><br/>(가) 기술집약 고기능 정밀산업 <br/><br/>
</p>
<p>휴대폰 등 모바일기기용 양면 테이프 산업은 영상부품소재 산업이고, 전방산업인 모바일을 연결시켜주는 중간재 산업인 동시에 고기능 정밀산업입니다. 휴대폰 부품은 세트가 요구하는 기능과 크기, 디자인과 출하시기 등을 충족시켜야 하므로 고기능 및정밀도를 갖추어야 합니다. 특히 휴대폰 산업은 기술진보가 매우 빠르고 제품의 수명주기가 짧기 때문에 수요자의 요구를 정확히 파악하여 신속히 대응하여야 합니다. </p>
<p><br/></p>
<p

In [393]:
desc = ''.join(bsobjt_final.findAll(text = True))
desc = desc.replace('\xa0', '').replace('\r', '').replace('\n', '').split('dtd"II. ')[1]
desc

"사업의 내용1. 사업의 개요가. 산업의 특성(1) 휴대폰 등 모바일기기용 양면 테이프(가) 기술집약 고기능 정밀산업 휴대폰 등 모바일기기용 양면 테이프 산업은 영상부품소재 산업이고, 전방산업인 모바일을 연결시켜주는 중간재 산업인 동시에 고기능 정밀산업입니다. 휴대폰 부품은 세트가 요구하는 기능과 크기, 디자인과 출하시기 등을 충족시켜야 하므로 고기능 및정밀도를 갖추어야 합니다. 특히 휴대폰 산업은 기술진보가 매우 빠르고 제품의 수명주기가 짧기 때문에 수요자의 요구를 정확히 파악하여 신속히 대응하여야 합니다.(나) 원가경쟁력과 품질의 중요성전방산업인 모바일의 고사양화 및 휴대폰 시장의 수요확대, 판가하락 추세에 따라 모바일 부품소재산업 내 경쟁의 핵심요소는 원가경쟁력 확보와 품질개선입니다. 글로벌 휴대폰 시장에서의 경쟁심화는 휴대폰의 단가 하락으로 이어지고 있으므로, 부품업체 입장에서 제품의 원가경쟁력은 필수적으로 갖추어야 합니다. 부품업체는 낮은 단가에서도 이익을 창출하면서 동시에 고품질을 유지하여야 합니다.(다) 고객사와의 긴밀한 공조체제휴대폰 등 전방산업의 라이프 사이클이 타 전자제품에 비해 상대적으로 짧고, 사용자들의 취향이 변하는 속도가 빠르기 때문에 타 산업에 비해 신속한 개발, 양산체제, 안정된 품질을 기반으로 고객사와 긴밀한 공조체제가 필요한 산업입니다.(2) 윈도우 필름 (가) 다양한 제품으로 구성 초기에는 제품의 색상 변질 방지가 중요한 이슈였고, 그 뒤에는 자외선 및 적외선의 차단이 중요해졌으며, 최근에는 실제적인 열차단 및 미관도 중요해지고 있습니다. 초기의 저가 제품부터 현재의 고가제품까지 다양한 제품들이 국가별/소비자별 선호에 따라서 판매되고 있습니다. 미국의 루마, 썬가드와 같은 선두브랜드가 있으며, 전세계적으로 수 많은 회사가 존재하고 있습니다.(나) 에너지 절감에 대한 관심 증가최근 각국의 전력부족이 심화되고 있고, 에너지 절감의 중요성이 커지면서 유해한 열에너지를 차단하고, 건물의 열효율성을 극대화 시킬 수 있는 건축용 IR필름

# 방림 (구분 O)

In [58]:
# 사업보고서 xpath를 통해 사업보고서 url 가지고 오기
elm = driver.find_element("xpath","/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a")
href = elm.get_attribute('href')

# input url 
html = urlopen(href) # 사업보고서 url
print(href)

https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210324000853


In [59]:
# 사업보고서 url로 반기보고서 html 호출
bsobjt = BeautifulSoup(html,"html.parser") 
# 기업명
title = str(bsobjt.find('title'))
title = title.split('\n\t')[1].split('/사업')[0]
# 사업의 내용
body = str(bsobjt.find('head'))
body = body.split('사업의 내용"')[1] # input2
body = body.split('cnt++')[0]
body = body.split(';')
body = [x for x in body if ('=' in x)]
body = [x.split('=')[1].split('"')[1].split('"')[0] for x in body]

In [60]:
# 사업내용 url 생성
url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[1] + '&dcmNo=' + body[2] + '&eleId=' + body[3] + '&offset=' + body[4] +'&length=' + body[5] + '&dtd=dart3.xsd'

In [61]:
# 사업의 내용 html 가지고 오기
html_final = urlopen(url_final)
bsobjt_final = BeautifulSoup(html_final,"html.parser")

In [62]:
bsobjt_final

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-1"><a name="toc1">II. 사업의 내용</a></p>
<p><span style="font-size:16pt;line-height:1.6em;color:#1600FF;font-weight:bold;">1. 사업의 개요</span><span style="font-weight:bold;"><br/></span><span style="color:#FF0000;font-weight:bold;"><br/></span><span>지배회사 및 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다. </span><span>지배회사인</span><span> <br/>(주)다우기술은 1986년 1월에 설립되어 소프트웨어와 시스템관련 솔루션의 개발 및 판매, 인터넷 관련 서비스 등을 주된 사업목적으로 하고 있습니다.  IT분야로 시작하여 2000년도에 국내최초 온라인 증권사인 키움증권을 설립하였으며, 이후 지속적인 투자를 통해 사업영역을 확장하고, 차별화된 기술력과 솔루션으로 국내를 대표하는 IT서비스 전문기업으로 위상을 차지하고 있습니다.

In [63]:
# 1. 사업의 개요, 2.주요 제품 및 서비스 추출

In [64]:
desc = ''.join(bsobjt_final.findAll(text = True))

In [65]:
desc.split('3. 주요 원재료')

['HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"\n\n\n\n\n\n\n\n\n\n\nII. 사업의 내용\n1. 사업의 개요지배회사 및 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다.\xa0지배회사인\xa0(주)다우기술은 1986년 1월에 설립되어 소프트웨어와 시스템관련 솔루션의 개발 및 판매, 인터넷 관련 서비스 등을 주된 사업목적으로 하고 있습니다.\xa0 IT분야로 시작하여 2000년도에 국내최초 온라인 증권사인 키움증권을 설립하였으며, 이후 지속적인 투자를 통해 사업영역을 확장하고, 차별화된 기술력과 솔루션으로 국내를 대표하는 IT서비스 전문기업으로 위상을 차지하고 있습니다. 연결종속기업 중 펀드 및 자산유동화 SPC를 제외한 국내 종속기업은 키움증권을 포함한 11개 회사이며, 해외 종속회사는 다우재팬 등 11개 입니다. 종속회사 중 상장회사는 1개(키움증권)입니다. (주)다우기술은 최상위 지배기업은 아니며, 최상위 지배기업은 다우데이타(032190)입니다..\n\n\n\n\n\n\n\n\n\n\n구분\n사업부문\n회사명\n주요 사업내용\n\n\n\n\n국내\nIT부문\n(주)다우기술(주)키다리스타\n소프트웨어 개발 및 판매, 시스템 구축 용역 및 인터넷 서비스 제공\n\n\n금융부문\n키움증권㈜ (상장)키움인베스트먼트㈜ 키움투자자산운용㈜ ㈜키움저축은행 ㈜키움예스저축은행키움프라이빗에쿼티 키움캐피탈키움에프앤아이\n\n국내 주식 및 파생상품 거래서비스, 벤처창업투자 전문 서비스투자 자문 및 종합자산운용서비스,상호저축은행업상호저축은행업전문사모집합투자업여신금융업NPL 투자관리업\n\n\n건물관리\n(주)키움이앤에스\xa0\n빌딩관리, 부동산컨설팅 등\n\n\n콘텐츠부문\n(주)키다리이엔티\n영화 투자 배급 부가판권 유통\n\n\n해외\nIT부문\n다우재팬다우홍콩\n소프트웨어 개발 및 판매\n\n\n금융부문\n키움증권 인도네시아키움자산운

In [66]:
len(desc.split('3. 주요 원재료'))

2

In [67]:
desc = desc.split('3. 주요 원재료')[0]

In [68]:
desc

'HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"\n\n\n\n\n\n\n\n\n\n\nII. 사업의 내용\n1. 사업의 개요지배회사 및 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다.\xa0지배회사인\xa0(주)다우기술은 1986년 1월에 설립되어 소프트웨어와 시스템관련 솔루션의 개발 및 판매, 인터넷 관련 서비스 등을 주된 사업목적으로 하고 있습니다.\xa0 IT분야로 시작하여 2000년도에 국내최초 온라인 증권사인 키움증권을 설립하였으며, 이후 지속적인 투자를 통해 사업영역을 확장하고, 차별화된 기술력과 솔루션으로 국내를 대표하는 IT서비스 전문기업으로 위상을 차지하고 있습니다. 연결종속기업 중 펀드 및 자산유동화 SPC를 제외한 국내 종속기업은 키움증권을 포함한 11개 회사이며, 해외 종속회사는 다우재팬 등 11개 입니다. 종속회사 중 상장회사는 1개(키움증권)입니다. (주)다우기술은 최상위 지배기업은 아니며, 최상위 지배기업은 다우데이타(032190)입니다..\n\n\n\n\n\n\n\n\n\n\n구분\n사업부문\n회사명\n주요 사업내용\n\n\n\n\n국내\nIT부문\n(주)다우기술(주)키다리스타\n소프트웨어 개발 및 판매, 시스템 구축 용역 및 인터넷 서비스 제공\n\n\n금융부문\n키움증권㈜ (상장)키움인베스트먼트㈜ 키움투자자산운용㈜ ㈜키움저축은행 ㈜키움예스저축은행키움프라이빗에쿼티 키움캐피탈키움에프앤아이\n\n국내 주식 및 파생상품 거래서비스, 벤처창업투자 전문 서비스투자 자문 및 종합자산운용서비스,상호저축은행업상호저축은행업전문사모집합투자업여신금융업NPL 투자관리업\n\n\n건물관리\n(주)키움이앤에스\xa0\n빌딩관리, 부동산컨설팅 등\n\n\n콘텐츠부문\n(주)키다리이엔티\n영화 투자 배급 부가판권 유통\n\n\n해외\nIT부문\n다우재팬다우홍콩\n소프트웨어 개발 및 판매\n\n\n금융부문\n키움증권 인도네시아키움자산운용

In [69]:
len(desc)

15170

In [70]:
desc = desc.replace('\xa0', '').replace('\r', '').replace('\n', '').split('HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"II. ')[1]
desc

'사업의 내용1. 사업의 개요지배회사 및 종속회사의 사업부문별 현황을 요약하면 아래와 같습니다.지배회사인(주)다우기술은 1986년 1월에 설립되어 소프트웨어와 시스템관련 솔루션의 개발 및 판매, 인터넷 관련 서비스 등을 주된 사업목적으로 하고 있습니다. IT분야로 시작하여 2000년도에 국내최초 온라인 증권사인 키움증권을 설립하였으며, 이후 지속적인 투자를 통해 사업영역을 확장하고, 차별화된 기술력과 솔루션으로 국내를 대표하는 IT서비스 전문기업으로 위상을 차지하고 있습니다. 연결종속기업 중 펀드 및 자산유동화 SPC를 제외한 국내 종속기업은 키움증권을 포함한 11개 회사이며, 해외 종속회사는 다우재팬 등 11개 입니다. 종속회사 중 상장회사는 1개(키움증권)입니다. (주)다우기술은 최상위 지배기업은 아니며, 최상위 지배기업은 다우데이타(032190)입니다..구분사업부문회사명주요 사업내용국내IT부문(주)다우기술(주)키다리스타소프트웨어 개발 및 판매, 시스템 구축 용역 및 인터넷 서비스 제공금융부문키움증권㈜ (상장)키움인베스트먼트㈜ 키움투자자산운용㈜ ㈜키움저축은행 ㈜키움예스저축은행키움프라이빗에쿼티 키움캐피탈키움에프앤아이국내 주식 및 파생상품 거래서비스, 벤처창업투자 전문 서비스투자 자문 및 종합자산운용서비스,상호저축은행업상호저축은행업전문사모집합투자업여신금융업NPL 투자관리업건물관리(주)키움이앤에스빌딩관리, 부동산컨설팅 등콘텐츠부문(주)키다리이엔티영화 투자 배급 부가판권 유통해외IT부문다우재팬다우홍콩소프트웨어 개발 및 판매금융부문키움증권 인도네시아키움자산운용 인도네시아Kiwoom Hong Kong LimitedKiwoom Singapore Pte. Ltd.Kiwoom BVI One LimitedKiwoom BVI Two LimitedKiwoom BVI Three Limited해외 현지 증권 및 자산운용서비스해외 현지 투자목적회사건물관리다우(대련)과기개발유한공사나미빌딩주차장관리유한공사해외 건물관리 및 부동산컨설팅주)연결대상 종속회사 중 실질적인 영업활동을 수행하지 